# Configuration

In [2]:

# --- Imports
import os
from pathlib import Path


import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Optional irsdk (for .ibt)
try:
    import irsdk
    IRSDK_AVAILABLE = True
except Exception:
    IRSDK_AVAILABLE = False

print(f"irsdk available   : {IRSDK_AVAILABLE}")

# --- Paths & Parameters
USE_IBT = True  # set True to read a .ibt with irsdk if available

# If USE_IBT=True, set your .ibt path:
ibt_file = Path("dataset_iracing_30_laps/mx5 mx52016_charlotte 2025 roval2025 2025-11-17 14-27-06.ibt")  # <--- change me if needed


# Laps to visualize in the "quick overlay" plots (only visualization; robust pipeline auto-selects reference)
laps_to_plot = [2, 3, 4]

# Core pipeline settings
N_SECTORS = 4               # number of sectors to split the track
BASE_GRID_LEN = 1000
TOPK_PERCENT = 0.30          # robust sector reference: median of fastest top K%
PACE_THRESHOLD_MS = 3000.0   # drop laps whose sum of positive sector losses > 2s
CLIP_MS = 1500.0             # clip training target (ms)
SEQ_LEN = 128                # per-sector resampling for stability
SEED = 42

# Output directory
OUT_DIR = Path("simulator_outputs")
OUT_DIR.mkdir(parents=True, exist_ok=True)


irsdk available   : True


# Dataset Loading

In [3]:

def load_from_ibt(ibt_path: Path) -> pd.DataFrame:
    if not IRSDK_AVAILABLE:
        raise RuntimeError("irsdk is not available in this environment.")
    if not ibt_path.exists():
        raise FileNotFoundError(f".ibt file not found: {ibt_path}")
    ibt = irsdk.IBT()
    try:
        ibt.open(str(ibt_path))
        channels = ['SessionTime','Lap','Speed','RPM','Throttle','Brake','LapDistPct','Lat','Lon','SteeringWheelAngle','Gear','LatAccel','LongAccel']
        data = {ch: ibt.get_all(ch) for ch in channels}
    finally:
        ibt.close()
    df = pd.DataFrame(data)
    return df

def basic_clean_and_units(df: pd.DataFrame) -> pd.DataFrame:
    df = df.copy()
    df.replace([np.inf, -np.inf], np.nan, inplace=True)
    df.dropna(inplace=True)
    if "Speed_KPH" not in df.columns and "Speed" in df.columns:
        df["Speed_KPH"] = df["Speed"] * 3.6
    if "Throttle_Pct" not in df.columns and "Throttle" in df.columns:
        df["Throttle_Pct"] = df["Throttle"] * 100.0
    if "Brake_Pct" not in df.columns and "Brake" in df.columns:
        df["Brake_Pct"] = df["Brake"] * 100.0
    return df

# --- Execute loading
if USE_IBT:
    print(f"Loading IBT: {ibt_file}")
    raw_df = load_from_ibt(ibt_file)
    df = basic_clean_and_units(raw_df)
    
else:
    raise RuntimeError("No data source selected.")

df.head()


Loading IBT: dataset_iracing_30_laps\mx5 mx52016_charlotte 2025 roval2025 2025-11-17 14-27-06.ibt


,SessionTime,Lap,Speed,RPM,Throttle,Brake,LapDistPct,Lat,Lon,SteeringWheelAngle,Gear,LatAccel,LongAccel,Speed_KPH,Throttle_Pct,Brake_Pct
0,289.883337,0,0.000000,300.0,0.0,1.0,0.000000,0.000000,0.000000,0.018981,0,-0.416223,-0.091603,0.000000,0.0,100.0
1,289.900004,0,0.007796,300.0,0.0,1.0,0.423894,35.349705,-80.685687,0.018981,0,0.821328,-0.045131,0.028067,0.0,100.0
2,289.916671,0,0.002468,300.0,0.0,1.0,0.423894,35.349705,-80.685687,0.018981,0,-0.604651,0.216608,0.008884,0.0,100.0
3,289.933337,0,0.019807,300.0,0.0,1.0,0.423894,35.349705,-80.685687,0.018981,0,1.372125,0.570752,0.071305,0.0,100.0
4,289.950004,0,0.043180,300.0,0.0,1.0,0.423894,35.349705,-80.685687,0.018981,0,0.675227,0.857857,0.155447,0.0,100.0
